## FAIR USE POLICY
**Please do not leave your Jupyter lab server idle for extended periods of time.** The Jupyter process, active Python kernels, and especially running Spark contexts, claim a minimum amount of cluster resources. These add up and will get starve resources of others eventually. Leaving your environment idle for a few hours (e.g., over lunch) is fine. But letting it idle overnight or for multiple days in which you are not actively using the cluster is not. You can kill the server from your SSH session, by pressing ctrl+c repeatedly, or by selecting *File->Shutdown* from the menu.

### Imports

In [1]:
import os
import time
from datetime import datetime, timedelta, date
import pprint

import collections
import numpy as np
import pandas as pd
import subprocess

import pyspark
# Find Spark
import findspark
findspark.init()


from datetime import datetime, timedelta, date
import datetime as dt
from datetime import timedelta as td
import os
import pandas as pd
import subprocess
import re
import pyarrow as pa
import copy
from pyspark.sql.types import DateType
from string import digits
from dateutil import parser

import nltk
import pyspark
from pyspark.sql import *  
from pyspark.sql.functions import *
import pyspark.sql.functions as F 
from pyspark.sql import functions as F, Window
from pyspark.sql.functions import from_unixtime
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import mean as _mean
from pyspark.sql.window import Window as W
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, LongType
from scipy import stats
from scipy.stats import mannwhitneyu



from functools import reduce
from pyspark.ml.feature import Tokenizer, StopWordsRemover


import json

#### Create Spark Configuration

In [2]:
# Create a SparkConf
APP_NAME = "apwg-var1-app"

spark_conf = pyspark.SparkConf().setAppName(APP_NAME).setMaster("yarn").set(
    "spark.submit.deployMode", "client"
).set("spark.sql.parquet.binaryAsString", "true"
).set("spark.dynamicAllocation.maxExecutors", "16"
).set("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp-spark24_2.11:3.0.0"
).set("spark.sql.debug.maxToStringFields", "1000"
).set("spark.executor.memory", "7G")

#### Start SparkContext
1. This may take a minute to complete
2. You should not (and cannot) start two Spark contexts. If you accidentally run this cell twice or get stuck somehow, restart your Python kernel from the menu above.
3. Please **stop your Spark context** when idling for extended periods of time (see code at bottom of notebook)

In [3]:
print("[{}] Starting Spark context.".format(datetime.now().replace(microsecond=0)))

# SparkContext
sc = pyspark.SparkContext(conf=spark_conf)

# SQLContext
sqlc = pyspark.SQLContext(sc)

[2021-04-25 19:28:31] Starting Spark context.


### Your code below

In [4]:

APWG_CLEAN_DATA_CONVERTED_BASE = "PATH TO DATA-RECORDS"

INTEREST_DATE_START = datetime(2017, 12, 18)
INTEREST_DATE_END   = datetime(2019, 8, 16)

# Read JSON files into Spark DF
clean_mails_df = sqlc.read.json(APWG_CLEAN_DATA_CONVERTED_BASE, multiLine=True).withColumn(
    "parsed_date", F.from_unixtime(F.col("date_received")).cast("date")
).filter(
    # Filter date range of interest
    (F.col("parsed_date") >= INTEREST_DATE_START.date().isoformat()) &
    (F.col("parsed_date") <= INTEREST_DATE_END.date().isoformat())
)

In [19]:
#filter emails in english language
original_df = clean_mails_df

In [20]:
original_df = original_df.filter(original_df.language == "english")

In [21]:
h1_df = original_df   
h1_df = h1_df.select('parsed_date', 'id')

In [22]:
remover = StopWordsRemover(inputCol='body_words', outputCol='words_clean')
h2_df = remover.transform(original_df)

h2_df = h2_df.drop("body_words")

word_list=['unsubscribed', 'hack', 'takedown', 'password', 'transparent',\
           'attempt', 'redirect', 'impersonate', 'network', 'obsolete', 'illegal', 'damage', 'edit',\
           'unauthenticated', 'initial', 'survey', 'collect', 'victim', 'detect', 'recharge', 'test',\
           'attachment', 'claim', 'profitable', 'virus', 'fraudulent', 'revalidation', 'link', 'description']

#array_intersect function requires two arrays as arguments, create array from the list of given values:
list_col = F.array(*[F.lit(cl) for cl in word_list])
h2_df = h2_df.filter(F.size(F.array_intersect(F.col("words_clean"), list_col)) > 0)
h2_df = h2_df.select('parsed_date', 'id')

In [23]:
h3_df = original_df.filter(original_df.email_has_attachments == "1")
h3_df = h3_df.select('parsed_date', 'id')

In [24]:
# announcement date of ddos attack

ddos_list = ['2018-01-17', '2018-01-27', '2018-02-08', '2018-03-01', '2018-03-06', '2018-05-14', '2018-05-24', '2018-07-30',\
             '2019-01-16','2019-01-31', '2019-02-23', '2019-03-22', '2019-04-16', '2019-06-12', '2019-08-08']


date_df = sqlc.createDataFrame([[d] for d in ddos_list], 'ddos_date string')


In [25]:
#partitionBy is used to shuffle data before applying the functions
def hypo_1(df, day):
    
    h1_df1 = (df.filter(f"parsed_date < '{day}' and parsed_date > '{day}' - interval 10 days")
             .withColumn('rn', F.dense_rank().over(Window.orderBy(F.desc('parsed_date'))))
             .filter('rn <= 3')
             .drop('rn')
             .withColumn('count_before', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )
  
    h1_df2 = (df.filter(f"parsed_date < '{day}' + interval 10 days and parsed_date > '{day}'")
             .withColumn('rn', F.dense_rank().over(Window.orderBy('parsed_date')))
             .filter('rn <= 3')
             .drop('rn')
             .withColumn('count_after', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )
   

    
    return [h1_df1, h1_df2]


In [26]:
def hypo_2(df, day):
    """
    Example usage: df_list = hypo_2(df, '2017-12-18', 15)
    Returns a list of 2 dataframes.
    """
    h2_df1 = (df.filter(f"parsed_date < '{day}' and parsed_date > '{day}' - interval 10 days")
             .withColumn('rn', F.dense_rank().over(Window.orderBy(F.desc('parsed_date'))))
             .filter('rn <= 7')
             .drop('rn')
             .withColumn('count_before', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )

    
    h2_df2 = (df.filter(f"parsed_date < '{day}' + interval 10 days and parsed_date > '{day}'")
             .withColumn('rn', F.dense_rank().over(Window.orderBy('parsed_date')))
             .filter('rn <= 7')
             .drop('rn')
             .withColumn('count_after', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )
   
    return [h2_df1, h2_df2]

In [27]:
def hypo_3(df, day):
    """
    Example usage: df_list = hypo_3(df, '2017-12-18', 15)
    Returns a list of 2 dataframes.
    """
    h3_df1 = (df.filter(f"parsed_date < '{day}' and parsed_date > '{day}' - interval 10 days")
             .withColumn('rn', F.dense_rank().over(Window.orderBy(F.desc('parsed_date'))))
             .filter('rn <= 7')
             .drop('rn')
             .withColumn('count_before', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )
  
    
    h3_df2 = (df.filter(f"parsed_date < '{day}' + interval 10 days and parsed_date > '{day}'")
             .withColumn('rn', F.dense_rank().over(Window.orderBy('parsed_date')))
             .filter('rn <= 7')
             .drop('rn')
             .withColumn('count_after', F.count('id').over(Window.partitionBy('parsed_date')))
             .orderBy('parsed_date').withColumn('ddos_date', F.lit(day))
          )
    
    
    
    return [h3_df1, h3_df2]

In [28]:
h1_df1_list = []
h1_df2_list = []

for d in ddos_list:
    h1_df1, h1_df2 = hypo_1(h1_df, d)
    
    h1_df1 = h1_df1.dropDuplicates(['parsed_date', 'count_before'])
    h1_df2 = h1_df2.dropDuplicates(['parsed_date', 'count_after'])
    
    h1_df1 = h1_df1.withColumn("sum_before", F.sum("count_before").over(Window.partitionBy("ddos_date")))
    h1_df2 = h1_df2.withColumn("sum_after", F.sum("count_after").over(Window.partitionBy("ddos_date")))
    
    
    h1_df1_list.append(h1_df1)
    h1_df2_list.append(h1_df2)
    
    
hypo1_df1 = reduce(DataFrame.unionAll, h1_df1_list)
hypo1_df2 = reduce(DataFrame.unionAll, h1_df2_list)


hypo1_df1 = (date_df.join(hypo1_df1, 'ddos_date', 'left')
                 .fillna(0, 'count_before')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_before').alias('sum_before'))
         )
hypo1_df2 = (date_df.join(hypo1_df2, 'ddos_date', 'left')
                 .fillna(0, 'count_after')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_after').alias('sum_after'))
         )

In [ ]:

hypo1_df1.show()
hypo1_df2.show()

''''''''''
expected output:
+----------+----------+
| ddos_date|sum_before|
+----------+----------+
|2018-01-17|      3084|
|2018-01-27|      3533|
|2018-02-08|      3427|
|2018-03-01|      3543|
|2018-03-06|      2917|
|2018-05-14|      2575|
|2018-05-24|      4174|
|2018-07-30|      3062|
|2019-01-16|         0|
|2019-01-31|         0|
|2019-02-23|      1679|
|2019-03-22|      2962|
|2019-04-16|      3470|
|2019-06-12|         0|
|2019-08-08|      4317|
+----------+----------+

+----------+---------+
| ddos_date|sum_after|
+----------+---------+
|2018-01-17|     3252|
|2018-01-27|     3405|
|2018-02-08|     2858|
|2018-03-01|     3122|
|2018-03-06|     2576|
|2018-05-14|     3806|
|2018-05-24|     3482|
|2018-07-30|     3891|
|2019-01-16|        0|
|2019-01-31|     1845|
|2019-02-23|     3150|
|2019-03-22|     7964|
|2019-04-16|     1870|
|2019-06-12|        0|
|2019-08-08|      266|
+----------+---------+

In [29]:
h2_df1_list = []
h2_df2_list = []

for d in ddos_list:
    h2_df1, h2_df2 = hypo_2(h2_df, d)
    
    h2_df1 = h2_df1.dropDuplicates(['parsed_date', 'count_before'])
    h2_df2 = h2_df2.dropDuplicates(['parsed_date', 'count_after'])
    
    h2_df1 = h2_df1.withColumn("sum_before", F.sum("count_before").over(Window.partitionBy("ddos_date")))
    h2_df2 = h2_df2.withColumn("sum_after", F.sum("count_after").over(Window.partitionBy("ddos_date")))
    
    h2_df1_list.append(h2_df1)
    h2_df2_list.append(h2_df2)
    
    
hypo2_df1 = reduce(DataFrame.unionAll, h2_df1_list)
hypo2_df2 = reduce(DataFrame.unionAll, h2_df2_list)

hypo2_df1 = (date_df.join(hypo2_df1, 'ddos_date', 'left')
                 .fillna(0, 'count_before')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_before').alias('sum_before'))
         )
hypo2_df2 = (date_df.join(hypo2_df2, 'ddos_date', 'left')
                 .fillna(0, 'count_after')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_after').alias('sum_after'))
         )
hypo2_df1.show()
hypo2_df2.show()

'''''''''''
expected output:

+----------+----------+
| ddos_date|sum_before|
+----------+----------+
|2018-01-17|      2964|
|2018-01-27|      2988|
|2018-02-08|      3013|
|2018-03-01|      2762|
|2018-03-06|      2846|
|2018-05-14|      2757|
|2018-05-24|      3387|
|2018-07-30|      3542|
|2019-01-16|         0|
|2019-01-31|         0|
|2019-02-23|       629|
|2019-03-22|      1200|
|2019-04-16|      2310|
|2019-06-12|         0|
|2019-08-08|      2103|
+----------+----------+

+----------+---------+
| ddos_date|sum_after|
+----------+---------+
|2018-01-17|     2897|
|2018-01-27|     3001|
|2018-02-08|     2969|
|2018-03-01|     3041|
|2018-03-06|     1241|
|2018-05-14|     3260|
|2018-05-24|     3548|
|2018-07-30|     3277|
|2019-01-16|        0|
|2019-01-31|      801|
|2019-02-23|     1297|
|2019-03-22|     7473|
|2019-04-16|      717|
|2019-06-12|        0|
|2019-08-08|      824|
+----------+---------+

In [30]:
h3_df1_list = []
h3_df2_list = []

for d in ddos_list:
    h3_df1, h3_df2 = hypo_3(h3_df, d)
    
    h3_df1 = h3_df1.dropDuplicates(['parsed_date', 'count_before'])
    h3_df2 = h3_df2.dropDuplicates(['parsed_date', 'count_after'])
    
    h3_df1 = h3_df1.withColumn("sum_before", F.sum("count_before").over(Window.partitionBy("ddos_date")))
    h3_df2 = h3_df2.withColumn("sum_after", F.sum("count_after").over(Window.partitionBy("ddos_date")))
    
    h3_df1_list.append(h3_df1)
    h3_df2_list.append(h3_df2)
    

hypo3_df1 = reduce(DataFrame.unionAll, h3_df1_list)
hypo3_df2 = reduce(DataFrame.unionAll, h3_df2_list)

hypo3_df1 = (date_df.join(hypo3_df1, 'ddos_date', 'left')
                 .fillna(0, 'count_before')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_before').alias('sum_before'))
         )
hypo3_df2 = (date_df.join(hypo3_df2, 'ddos_date', 'left')
                 .fillna(0, 'count_after')
                 .groupBy('ddos_date')
                 .agg(F.sum('count_after').alias('sum_after'))
         )
hypo3_df1.show()
hypo3_df2.show()

'''''''''''
expected output:

+----------+----------+
| ddos_date|sum_before|
+----------+----------+
|2018-01-17|       992|
|2018-01-27|      1028|
|2018-02-08|      1066|
|2018-03-01|       839|
|2018-03-06|      1148|
|2018-05-14|       887|
|2018-05-24|      1016|
|2018-07-30|      1168|
|2019-01-16|         0|
|2019-01-31|         0|
|2019-02-23|       273|
|2019-03-22|       187|
|2019-04-16|       813|
|2019-06-12|         0|
|2019-08-08|       470|
+----------+----------+

+----------+---------+
| ddos_date|sum_after|
+----------+---------+
|2018-01-17|     1031|
|2018-01-27|     1186|
|2018-02-08|     1037|
|2018-03-01|     1269|
|2018-03-06|      450|
|2018-05-14|      859|
|2018-05-24|     1208|
|2018-07-30|      927|
|2019-01-16|        0|
|2019-01-31|      285|
|2019-02-23|      385|
|2019-03-22|     1253|
|2019-04-16|      268|
|2019-06-12|        0|
|2019-08-08|      118|
+----------+---------+


#### Stop the SparkContext
note: don't run this block unless you actually want to stop your context

In [36]:
print("[{}] Stopping Spark context.".format(datetime.now().replace(microsecond=0)))
sc.stop()

[2021-04-26 01:27:41] Stopping Spark context.
